In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import gdal

In [2]:
# load cells polygons
geo_cells = gpd.read_file(r'data\cells_polygons_california_only.shp',crs="EPSG:4326")
geo_cells.head()

geo_cells = geo_cells.drop(columns=['path','layer'])
geo_cells.head()

,fid,location,geometry
0,1.0,C:/Users/Dell/OneDrive - Technion/××× ×××...,"POLYGON ((-115.00042 33.00042, -114.90042 33.0..."
1,2.0,C:/Users/Dell/OneDrive - Technion/××× ×××...,"POLYGON ((-114.90042 33.00042, -114.80042 33.0..."
2,3.0,C:/Users/Dell/OneDrive - Technion/××× ×××...,"POLYGON ((-114.80042 33.00042, -114.70042 33.0..."
3,4.0,C:/Users/Dell/OneDrive - Technion/××× ×××...,"POLYGON ((-114.70042 33.00042, -114.60042 33.0..."
4,5.0,C:/Users/Dell/OneDrive - Technion/××× ×××...,"POLYGON ((-114.60042 33.00042, -114.50042 33.0..."


In [3]:
# load fires data
fires = gpd.read_file(r'data\fires_2012-15.shp',crs="EPSG:4326")
fires.head()

,Unnamed_ 0,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYS,SOURCE_S_1,NWCG_REPOR,NWCG_REP_1,NWCG_REP_2,SOURCE_REP,...,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESC,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape,geometry
0,1567432,1567433,201433714,FS-1503920,FED,FS-FIRESTAT,FS,USCAANF,Angeles National Forest,0501,...,34.204167,-117.808333,5.0,USFS,CA,037,37.0,Los Angeles,b'\x00\x01\xad\x10\x00\x00x\'\xb8\xbb\xbbs]\xc...,POINT (-117.80833 34.20417)
1,1567433,1567434,201433715,FS-1507944,FED,FS-FIRESTAT,FS,USCAANF,Angeles National Forest,0501,...,34.345278,-117.928889,5.0,USFS,CA,037,37.0,Los Angeles,b'\x00\x01\xad\x10\x00\x00\xb8\nc\xear{]\xc0`\...,POINT (-117.92889 34.34528)
2,1567434,1567435,201433716,FS-1510402,FED,FS-FIRESTAT,FS,USCAANF,Angeles National Forest,0501,...,34.548056,-118.671667,5.0,USFS,CA,037,37.0,Los Angeles,b'\x00\x01\xad\x10\x00\x00\xa4]3\x96\xfc\xaa]\...,POINT (-118.67167 34.54806)
3,1567435,1567436,201433717,FS-1503567,FED,FS-FIRESTAT,FS,USCAANF,Angeles National Forest,0501,...,34.174722,-117.672500,5.0,USFS,CA,071,71.0,San Bernardino,b'\x00\x01\xad\x10\x00\x00\xd4\xa3p=\nk]\xc0pd...,POINT (-117.67250 34.17472)
4,1567436,1567437,201433718,FS-1504047,FED,FS-FIRESTAT,FS,USCAANF,Angeles National Forest,0501,...,34.489444,-118.285833,14.0,MISSING/NOT SPECIFIED,CA,037,37.0,Los Angeles,b'\x00\x01\xad\x10\x00\x008\x1d\xe1\x17K\x92]\...,POINT (-118.28583 34.48944)


In [4]:
# calculate topographic statistics for each cell
for i, cell in geo_cells.iterrows():
    path = str(cell['location'][str.find(cell['location'],'data'):])
    raster = gdal.Open(path)
    band = raster.GetRasterBand(1)
    stats = band.ComputeStatistics(0)
    geo_cells.at[i,'min_height'] = stats[0]
    geo_cells.at[i,'max_height'] = stats[1]
    geo_cells.at[i,'avg_height'] = stats[2]
    geo_cells.at[i,'std_height'] = stats[3]

geo_cells.head()

,fid,location,geometry,min_height,max_height,avg_height,std_height
0,1.0,C:/Users/Dell/OneDrive - Technion/××× ×××...,"POLYGON ((-115.00042 33.00042, -114.90042 33.0...",91.0,214.0,148.968819,25.747558
1,2.0,C:/Users/Dell/OneDrive - Technion/××× ×××...,"POLYGON ((-114.90042 33.00042, -114.80042 33.0...",141.0,463.0,205.685903,34.354413
2,3.0,C:/Users/Dell/OneDrive - Technion/××× ×××...,"POLYGON ((-114.80042 33.00042, -114.70042 33.0...",165.0,592.0,246.467083,33.330296
3,4.0,C:/Users/Dell/OneDrive - Technion/××× ×××...,"POLYGON ((-114.70042 33.00042, -114.60042 33.0...",61.0,521.0,230.628333,47.022054
4,5.0,C:/Users/Dell/OneDrive - Technion/××× ×××...,"POLYGON ((-114.60042 33.00042, -114.50042 33.0...",48.0,400.0,164.990556,49.978685


In [11]:
geo_cells.to_file('geo_cells_basic.shp')

create cell for every week in every year

In [29]:
geo_cells_with_weeks = geo_cells.copy()
geo_cells_with_weeks['week'] = 39
geo_cells_with_weeks['year'] = 2012

In [30]:
# for 2012

for i in range(40,52):
    gd = geo_cells.copy()
    gd['week'] = i
    gd['year'] = 2012
    geo_cells_with_weeks = geo_cells_with_weeks.append(gd)

In [31]:
# for 2013-2015

for j in range (2013,2016):
    for i in range(0,52):
        gd = geo_cells.copy()
        gd['week'] = i
        gd['year'] = j
        geo_cells_with_weeks = geo_cells_with_weeks.append(gd)

geo_cells_with_weeks.reset_index(drop=True, inplace=True)

In [33]:
# fitting fires to cells in year 2012
for i in range(39,52):
    temp_fires = fires[fires['FIRE_YEAR']==2012]
    temp_fires = temp_fires[fires['DISCOVER_1']//7==i]
    for j, row in geo_cells.iterrows():
        pip = temp_fires.within((geo_cells.loc[j,'geometry'])) # check if inside cell
        fire_in_cell = temp_fires[pip==True]
        if fire_in_cell.shape[0] >0 : # case of fire in cell
            index = j + (i-39)*geo_cells.shape[0] # finding the index of the right cell
            geo_cells_with_weeks.at[index,'fire'] = 1

C:\Users\Dell\Anaconda3\lib\site-packages\geopandas\geodataframe.py:546: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


In [34]:
# fitting fires to cells in years 2013-2015
for y in range(2013,2016):
    for i in range(52):
        temp_fires = fires[fires['FIRE_YEAR']==y]
        temp_fires = temp_fires[fires['DISCOVER_1']//7==i]
        for j, row in geo_cells.iterrows():
            pip = temp_fires.within((geo_cells.loc[j,'geometry'])) # check if inside cell
            fire_in_cell = temp_fires[pip==True]
            if fire_in_cell.shape[0] >0 : # case of fire in cell
                index = j + i*geo_cells.shape[0] + 13*geo_cells.shape[0] + ((y-2013)*52*geo_cells.shape[0]) # finding the index of the right cell
                geo_cells_with_weeks.at[index,'fire'] = 1

In [35]:
geo_cells_with_weeks.to_file('geo_cells_complete.shp')

In [2]:
geo_cells_with_weeks = gpd.read_file('geo_cells_complete.shp')